# Exploring The Emerald City(Sydney)

### Importing the libraries  

In [4]:
import pandas as pd # Library to handle Data Analytics
import numpy as np # Library to handle Vectorized Data
from sklearn.cluster import KMeans # Library for Kmeans for Clustering Stage
import json # To handle json files
from pandas.io.json import json_normalize # transfrom json files into dataframe
import matplotlib.cm as cm # matplotlib and it's modules
import matplotlib.colors as colors
import urllib.request # to handle url requests
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # to get the coordinates of a city
import folium #to generate interactive maps
import requests # to handle requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

Getting the coordinates of sydney using geopy

In [5]:
address='Sydney'
geolocator=Nominatim(user_agent='sydney_explorer')
location=geolocator.geocode(address) # it gives us the location details
latitude=location.latitude # extracting latitude value from location
longitude=location.longitude # extracting longitude value from location
print('latitude and longitude coordinates of sydney are{},{}:',latitude,longitude)

latitude and longitude coordinates of sydney are{},{}: -33.8548157 151.2164539


Foursquare Credentials

In [6]:
CLIENT_ID = 'RAOF0N5DNV32EYBC0GAAW23DS3FCDDI0X4QUZ1FHNBI5OJQJ' # your Foursquare ID
CLIENT_SECRET = '4F3LQVP4N2HAUUXEB5WCN53UHJL5DQO1UYFB3BZYSJE5GWXQ' # your Foursquare Secret
VERSION = '20200503' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RAOF0N5DNV32EYBC0GAAW23DS3FCDDI0X4QUZ1FHNBI5OJQJ
CLIENT_SECRET:4F3LQVP4N2HAUUXEB5WCN53UHJL5DQO1UYFB3BZYSJE5GWXQ


In [7]:
LIMIT=200
radius=1500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,latitude,longitude,radius,LIMIT)
print(url)


https://api.foursquare.com/v2/venues/explore?&client_id=RAOF0N5DNV32EYBC0GAAW23DS3FCDDI0X4QUZ1FHNBI5OJQJ&client_secret=4F3LQVP4N2HAUUXEB5WCN53UHJL5DQO1UYFB3BZYSJE5GWXQ&v=20200503&ll=-33.8548157,151.2164539&radius=1500&limit=200


In [8]:
results1=requests.get(url).json() # storing the results into a variable 


In [9]:
results1

{'meta': {'code': 200, 'requestId': '5eccc0aaaba297001b13017a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 203,
  'suggestedBounds': {'ne': {'lat': -33.84131568649999,
    'lng': 151.2326797739707},
   'sw': {'lat': -33.86831571350002, 'lng': 151.20022802602932}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '502d0f6fe4b047ef99541af2',
       'name': 'Sydney Opera House - Concert Hall',
       'location': {'address': '2 Macquarie St',
        'lat': -33.856594787181294,
        'lng': 151.21505812823835,
        'labeledLatLngs': [{'label': 'display',
 

## cleaning the data

In [10]:
def get_category(row): # function which returns category details 
    try:
        category_list=row['categories']
    except:
        category_list=row['venue.categories']
    if len(category_list)==0:
        return None
    else:
        return category_list[0]['name']

Transforming the json data into data frame

In [11]:
venues=results1['response']['groups'][0]['items'] # getting the venue details from json file into a variable.
df1=json_normalize(venues) # transforming the json data into data frame.
columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng'] # creating a list columns.
sydney=df1.loc[:,columns] # adding the columns list to data frame.
sydney['venue.category']=sydney.apply(get_category,axis=1) # adding venue column into data frame.
sydney.columns=[i.split('.')[-1]  for i in sydney]
sydney.drop(['categories'],axis=1,inplace=True)


In [12]:
sydney.head(20)

,name,lat,lng,category
0,Sydney Opera House - Concert Hall,-33.856595,151.215058,Concert Hall
1,Drama Theatre,-33.856863,151.214759,Theater
2,Sydney Opera House,-33.857260,151.215040,Opera House
3,Sydney Opera House - Studio,-33.856991,151.214482,Theater
4,Sydney Opera House - Playhouse,-33.857518,151.214559,Theater
5,Opera Kitchen,-33.857972,151.213989,Café
6,Opera Bar,-33.858409,151.213976,Cocktail Bar
7,Joan Sutherland Theatre,-33.857514,151.214286,Concert Hall
8,Sydney Opera House Forecourt,-33.858184,151.214619,Plaza
9,Sydney Harbour Bridge,-33.852289,151.210681,Bridge


In [13]:
sydney.category.unique() # finding the no of unique categories in the data frame.

array(['Concert Hall', 'Theater', 'Opera House', 'Café', 'Cocktail Bar',
       'Plaza', 'Bridge', 'Australian Restaurant', 'Scenic Lookout',
       'Trail', 'Hotel', 'Monument / Landmark', 'History Museum',
       'Japanese Restaurant', 'Tour Provider', 'Hotel Bar', 'Park',
       'Burger Joint', 'Art Museum', 'Pub', 'Botanical Garden', 'Garden',
       'Brewery', 'Bakery', 'Ice Cream Shop', 'Movie Theater',
       'Performing Arts Venue', 'Pool', 'Indie Movie Theater', 'Tree',
       'Hostel', 'Lounge', 'Thai Restaurant', 'Italian Restaurant',
       'Food Court', 'Chinese Restaurant', 'Indian Restaurant',
       'Beer Garden', 'Harbor / Marina', 'Planetarium', 'Music School',
       'Speakeasy', 'French Restaurant', 'Yoga Studio', 'Library',
       'Asian Restaurant', 'Seafood Restaurant', 'Coffee Shop',
       'Steakhouse'], dtype=object)

In [14]:
print('no of unique categories are {}.'.format(len(sydney['category'].unique())))

no of unique categories are 49.


In [15]:
l=list(sydney['category']) # getting the category details into a list.
l

['Concert Hall',
 'Theater',
 'Opera House',
 'Theater',
 'Theater',
 'Café',
 'Cocktail Bar',
 'Concert Hall',
 'Plaza',
 'Bridge',
 'Australian Restaurant',
 'Scenic Lookout',
 'Scenic Lookout',
 'Scenic Lookout',
 'Trail',
 'Hotel',
 'Monument / Landmark',
 'Australian Restaurant',
 'Australian Restaurant',
 'Australian Restaurant',
 'Scenic Lookout',
 'Café',
 'History Museum',
 'Japanese Restaurant',
 'Australian Restaurant',
 'Tour Provider',
 'Hotel Bar',
 'Park',
 'Burger Joint',
 'Art Museum',
 'Pub',
 'Park',
 'Botanical Garden',
 'Garden',
 'Brewery',
 'Bakery',
 'Cocktail Bar',
 'Trail',
 'Scenic Lookout',
 'Cocktail Bar',
 'Ice Cream Shop',
 'Movie Theater',
 'Performing Arts Venue',
 'Pool',
 'Café',
 'Pool',
 'Indie Movie Theater',
 'Hotel',
 'Café',
 'Tree',
 'Café',
 'Hostel',
 'Park',
 'Lounge',
 'Thai Restaurant',
 'Australian Restaurant',
 'Italian Restaurant',
 'Café',
 'Hotel Bar',
 'Scenic Lookout',
 'Pool',
 'Food Court',
 'Hotel Bar',
 'Theater',
 'Hotel Bar',


Adding new column category type.<br>
They are 52 unique categories in the data frame so to make it simple reduce the categories by grouping them based on the categories into a new column

In [16]:
for category in range(len(l)):
    if   l[category] in ['Café','Coffee Shop']: 
         l[category]='coffee shop'
    elif l[category] in ['Seafood Restaurant','Hotel','Restaurant', 'French Restaurant','Bistro','Asian Restaurant','Middle Eastern Restaurant','Mexican Restaurant','Indian Restaurant','Australian Restaurant','Chinese Restaurant','Tapas Restaurant','Thai Restaurant','Japanese Restaurant','Italian Restaurant']:
         l[category]='Restaurant'
    elif l[category] in ['Speakeasy','Bar','Hotel Bar','Beer Garden','Cocktail Bar', 'Brewery','Pub']:
         l[category]='Bar'
    elif l[category] in ['Pizza Place','Food Court','Burger Joint','Food & Drink Shop',]:
         l[category]='Fast Food'
    elif l[category] in ['Garden','Planetarium','Tree','Botanical Garden','Museum','Art Museum','History Museum','Tour Provider','Scenic Lookout','Monument / Landmark','Bridge','Opera House','Trail','Harbor / Marina']:
         l[category]='Tourist Spots'
    elif l[category] in ['Theater','Movie Theater','Concert Hall','Indie Movie Theater']:
         l[category]='Theatre'
    elif l[category] in ['Park','Pool','Theme Park']:
         l[category]='Entertainment'
    elif l[category] in ['Creperie','Bakery','Dessert Shop','Ice Cream Shop']:
         l[category] ='Desserts'
    elif l[category] in ['Plaza','Flea Market','Steakhouse','Steak']:
         l[category]='Retail Store and Markets'
    elif l[category] in ['Lounge','Hostel']:
         l[category]='Accomodation'
    elif l[category] in ['Library','Music School','Yoga Studio','Performing Arts Venue']:
         l[category]='Fitness'
            

In [17]:
sydney['category type']=l # adding the new the column 

In [18]:
sydney.head(20)

,name,lat,lng,category,category type
0,Sydney Opera House - Concert Hall,-33.856595,151.215058,Concert Hall,Theatre
1,Drama Theatre,-33.856863,151.214759,Theater,Theatre
2,Sydney Opera House,-33.857260,151.215040,Opera House,Tourist Spots
3,Sydney Opera House - Studio,-33.856991,151.214482,Theater,Theatre
4,Sydney Opera House - Playhouse,-33.857518,151.214559,Theater,Theatre
5,Opera Kitchen,-33.857972,151.213989,Café,coffee shop
6,Opera Bar,-33.858409,151.213976,Cocktail Bar,Bar
7,Joan Sutherland Theatre,-33.857514,151.214286,Concert Hall,Theatre
8,Sydney Opera House Forecourt,-33.858184,151.214619,Plaza,Retail Store and Markets
9,Sydney Harbour Bridge,-33.852289,151.210681,Bridge,Tourist Spots


Using onehot encoding to get the dummy values for category_type.

In [19]:
sydney_onehot1=pd.get_dummies(sydney[['category type']],prefix='',prefix_sep='')
sydney_onehot1.head(10)

,Accomodation,Bar,Desserts,Entertainment,Fast Food,Fitness,Restaurant,Retail Store and Markets,Theatre,Tourist Spots,coffee shop
0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,1
6,0,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0,1,0


finding out the mean of the categories

In [20]:
sydney_1=sydney_onehot1.mean()
sydney_1

Accomodation                0.02
Bar                         0.15
Desserts                    0.04
Entertainment               0.09
Fast Food                   0.03
Fitness                     0.04
Restaurant                  0.21
Retail Store and Markets    0.02
Theatre                     0.08
Tourist Spots               0.22
coffee shop                 0.10
dtype: float64

Transforming the mean values into an array. 

In [21]:
onehot_arr=np.array(sydney_1) # creating an array for mean values of category_type

In [22]:
res=onehot_arr.reshape(-1,1) # reshaping the data into 2d array.
res.shape

(11, 1)

## Clustering

In [23]:
kcluster=11 # no of clusters.
kmeans=KMeans(n_clusters=kcluster,random_state=0).fit(sydney_onehot1) # initialising the cluster algorithm and fitting the data.
kmeans.labels_[0:10] # creating the cluster labels.

array([5, 5, 1, 5, 5, 0, 4, 5, 9, 1], dtype=int32)

In [24]:
sydney.insert(3,'Cluster_Labels',kmeans.labels_,True) # adding the cluster labels to the data frame

### Cluster-1 

In [25]:
cluster1=sydney.loc[sydney['Cluster_Labels']==0] # cluster1

In [26]:
cluster1 # it contains about all coffee shops in the city

,name,lat,lng,Cluster_Labels,category,category type
5,Opera Kitchen,-33.857972,151.213989,0,Café,coffee shop
21,The Tea Cosy,-33.857413,151.208561,0,Café,coffee shop
44,The Fine Food Store,-33.858546,151.208633,0,Café,coffee shop
48,Café Opera,-33.863089,151.212463,0,Café,coffee shop
50,Cabrito Coffee Traders,-33.862516,151.209324,0,Café,coffee shop
57,Celsius Coffee Co.,-33.849403,151.220256,0,Café,coffee shop
67,Bertoni,-33.865167,151.211522,0,Café,coffee shop
73,Creperie Suzette,-33.859534,151.207994,0,Café,coffee shop
88,Marlowe's Way,-33.863877,151.208744,0,Café,coffee shop
90,Georgie Boy's Coffee Co.,-33.862403,151.207591,0,Coffee Shop,coffee shop


### Cluster-2

In [27]:
cluster2=sydney.loc[sydney['Cluster_Labels']==1]

In [28]:
cluster2 # it contains about all tourist spots.

,name,lat,lng,Cluster_Labels,category,category type
2,Sydney Opera House,-33.857260,151.215040,1,Opera House,Tourist Spots
9,Sydney Harbour Bridge,-33.852289,151.210681,1,Bridge,Tourist Spots
11,Kirribilli Lookout,-33.849323,151.213298,1,Scenic Lookout,Tourist Spots
12,Harbour Bridge Pylon Lookout,-33.854580,151.209492,1,Scenic Lookout,Tourist Spots
13,Mrs. Macquarie's Point,-33.859395,151.222204,1,Scenic Lookout,Tourist Spots
14,The Opera House to the Botanic Gardens Walk,-33.860914,151.213221,1,Trail,Tourist Spots
16,Mrs Macquarie's Chair,-33.859534,151.222539,1,Monument / Landmark,Tourist Spots
20,Dawes Point,-33.855194,151.209820,1,Scenic Lookout,Tourist Spots
22,Government House,-33.860616,151.214903,1,History Museum,Tourist Spots
25,BridgeClimb Sydney,-33.857518,151.207832,1,Tour Provider,Tourist Spots


### Cluster-3

In [29]:
cluster3=sydney.loc[sydney['Cluster_Labels']==2]

In [30]:
cluster3 # it contains all parks and pools in the city. 

,name,lat,lng,Cluster_Labels,category,category type
27,Bradfield Park,-33.848728,151.212132,2,Park,Entertainment
31,Dr Mary Booth Park & Lookout,-33.850418,151.215772,2,Park,Entertainment
43,Rooftop Pool (Park Hyatt),-33.855876,151.209386,2,Pool,Entertainment
45,Andrew (Boy) Charlton Pool,-33.863664,151.221581,2,Pool,Entertainment
52,Observatory Hill,-33.859125,151.204977,2,Park,Entertainment
60,North Sydney Olympic Pool,-33.849320,151.210886,2,Pool,Entertainment
70,Blues Point Reserve,-33.848932,151.204026,2,Park,Entertainment
84,Barangaroo Reserve,-33.857052,151.201100,2,Park,Entertainment
94,The Domain,-33.867868,151.215370,2,Park,Entertainment


### Cluster-4

In [31]:
cluster4=sydney.loc[sydney['Cluster_Labels']==3]

In [32]:
cluster4 # it contains all the restaurants in the city

,name,lat,lng,Cluster_Labels,category,category type
10,Bennelong Restaurant,-33.857564,151.214794,3,Australian Restaurant,Restaurant
15,Park Hyatt Sydney,-33.856023,151.209225,3,Hotel,Restaurant
17,Aria Restaurant,-33.859198,151.213261,3,Australian Restaurant,Restaurant
18,Quay,-33.857445,151.210155,3,Australian Restaurant,Restaurant
19,Portside Sydney,-33.857050,151.214650,3,Australian Restaurant,Restaurant
23,Yukis At The Quay,-33.858409,151.210287,3,Japanese Restaurant,Restaurant
24,6 Head 1788,-33.856670,151.209150,3,Australian Restaurant,Restaurant
47,InterContinental Sydney,-33.863076,151.212066,3,Hotel,Restaurant
54,Chat Thai | ชาติไทย (Chat Thai),-33.861872,151.209958,3,Thai Restaurant,Restaurant
55,Café Sydney,-33.862217,151.210934,3,Australian Restaurant,Restaurant


### Cluster-5

In [33]:
cluster5=sydney.loc[sydney['Cluster_Labels']==4]
cluster5 # it contains all the bars in the city.

,name,lat,lng,Cluster_Labels,category,category type
6,Opera Bar,-33.858409,151.213976,4,Cocktail Bar,Bar
26,The Bar at Park Hyatt Sydney,-33.856937,151.209319,4,Hotel Bar,Bar
30,The Glenmore Rooftop Hotel,-33.858632,151.207464,4,Pub,Bar
34,The Squire's Landing,-33.857346,151.210219,4,Brewery,Bar
36,Bulletin Place,-33.862344,151.209339,4,Cocktail Bar,Bar
39,Garden Bar by the Corner House,-33.857113,151.214437,4,Cocktail Bar,Bar
58,The Cortile,-33.862977,151.212463,4,Hotel Bar,Bar
62,Club Intercontinental,-33.862820,151.212489,4,Hotel Bar,Bar
64,Blu Bar On 36,-33.861067,151.206361,4,Hotel Bar,Bar
71,Busby's Bar,-33.862598,151.220418,4,Beer Garden,Bar


### Cluster-6

In [34]:
cluster6=sydney.loc[sydney['Cluster_Labels']==5]
cluster6 # it contains all theatres in the city.

,name,lat,lng,Cluster_Labels,category,category type
0,Sydney Opera House - Concert Hall,-33.856595,151.215058,5,Concert Hall,Theatre
1,Drama Theatre,-33.856863,151.214759,5,Theater,Theatre
3,Sydney Opera House - Studio,-33.856991,151.214482,5,Theater,Theatre
4,Sydney Opera House - Playhouse,-33.857518,151.214559,5,Theater,Theatre
7,Joan Sutherland Theatre,-33.857514,151.214286,5,Concert Hall,Theatre
41,St. George OpenAir Cinema,-33.861503,151.220864,5,Movie Theater,Theatre
46,Dendy Cinemas,-33.859505,151.212910,5,Indie Movie Theater,Theatre
63,Roslyn Packer Theatre,-33.857019,151.204947,5,Theater,Theatre


### Cluster-7

In [35]:
cluster7=sydney.loc[sydney['Cluster_Labels']==6]
cluster7 # it contains all fast food centers that are available in the city

,name,lat,lng,Cluster_Labels,category,category type
42,Sydney Theatre Company,-33.857028,151.204938,6,Performing Arts Venue,Fitness
77,Sydney Conservatorium of Music,-33.863457,151.213779,6,Music School,Fitness
85,Yoga Beyond,-33.863768,151.221375,6,Yoga Studio,Fitness
86,State Library of New South Wales,-33.866693,151.212483,6,Library,Fitness


### Cluster-8

In [36]:
cluster8=sydney.loc[sydney['Cluster_Labels']==7]
cluster8 # it contains all the fitness,libray and music school in the city. 

,name,lat,lng,Cluster_Labels,category,category type
35,La Renaissance Café Pâtisserie,-33.859260,151.208428,7,Bakery,Desserts
40,Gelato Messina,-33.862090,151.209860,7,Ice Cream Shop,Desserts
81,Gelatissimo,-33.861283,151.211651,7,Ice Cream Shop,Desserts
98,1 Abercrombie Lane,-33.864039,151.207502,7,Bakery,Desserts


### Cluster-9

In [37]:
cluster9=sydney.loc[sydney['Cluster_Labels']==8]
cluster9 # it contains all the dessert shops in the city.

,name,lat,lng,Cluster_Labels,category,category type
28,Ribs & Burgers,-33.857753,151.209045,8,Burger Joint,Fast Food
61,Gateway Foodcourt,-33.862051,151.209879,8,Food Court,Fast Food
65,Bar Luca,-33.864078,151.211626,8,Burger Joint,Fast Food


### Cluster-10

In [38]:
cluster10=sydney.loc[sydney['Cluster_Labels']==9]
cluster10 # it contains about markets available int he 

,name,lat,lng,Cluster_Labels,category,category type
8,Sydney Opera House Forecourt,-33.858184,151.214619,9,Plaza,Retail Store and Markets
93,Rockpool Bar & Grill,-33.866206,151.210120,9,Steakhouse,Retail Store and Markets


### Cluster-11

In [39]:
cluster11=sydney.loc[sydney['Cluster_Labels']==10]
cluster11

,name,lat,lng,Cluster_Labels,category,category type
51,Sydney Harbour YHA,-33.860128,151.206844,10,Hostel,Accomodation
53,The Botanist,-33.845386,151.212772,10,Lounge,Accomodation


### Sydney Map

In [40]:
sydney_map=folium.Map(location=[latitude,longitude],zoom_start=14)
sydney_map

In [44]:
x=np.arange(kcluster)
y=[i+x+(i*x)**2 for i in range(kcluster)]
color_array=cm.rainbow(np.linspace(0,1,len(y)))
rainbow=[colors.rgb2hex(i) for i in color_array]
for lat,lon,cluster,name,category_type in zip(sydney['lat'],sydney['lng'],sydney['Cluster_Labels'],sydney['name'],sydney['category type']):
    label=folium.Popup('Name:'+str(name)+','+'Category:'+str(category_type),parse_html=True)
    folium.CircleMarker([lat,lon],
                        radius=3,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.8).add_to(sydney_map)
sydney_map